<a href="https://colab.research.google.com/github/HurtaClaudio/Coursera_Capstone/blob/main/BERTopic_extracci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/BERTopic/BERTopic'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/BERTopic/BERTopic


In [ ]:
%%capture
!pip install bertopic
!pip install --upgrade tbb
!pip install openpyxl==3.0.9

# Nueva sección

In [ ]:
import pandas as pd
import numpy as np
import pickle
from hdbscan import HDBSCAN

In [ ]:
try:
    from bertopic import BERTopic
except Exception:
    pass

try:
    from bertopic import BERTopic
except Exception:
    pass

try:
    from umap import UMAP
except Exception:
    pass

try:
    from umap import UMAP
except Exception:
    pass

In [ ]:
df = pd.read_excel('bbdd.xlsx')
len(df)
df.head(5)

,REC_PERIODO,EMPRESA,FOLIO,MERCADOAGRUPADO,REC_CIA,VERBATIM_PROBLEMAS
0,202110,ENTEL,32844415,POSTPAGO,0.0,"de internet, esta muy lento, cada vez que lo n..."
1,202110,ENTEL,32847714,POSTPAGO,0.0,"que me estaban haciendo cobro indebido, si, ca..."
2,202110,ENTEL,32848155,POSTPAGO,0.0,me cobran de más en el plan porque yo pago 9 m...
3,202110,ENTEL,32851851,POSTPAGO,-100.0,el internet funciona muy lento y la señal igua...
4,202110,ENTEL,32851998,PREPAGO,-100.0,constante caída de su aplicación lo informe me...


In [ ]:
df = df[df.VERBATIM_PROBLEMAS.isnull() == False]
df = df[df.MERCADOAGRUPADO == 'POSTPAGO']
df.reset_index(drop = True, inplace= True)
len(df)

4310

In [ ]:
docs = list(df['VERBATIM_PROBLEMAS'])

In [ ]:
umap_model = UMAP(n_neighbors=15, n_components=10, 
                  min_dist=0.0, metric='cosine', random_state=41)
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=5)

In [ ]:
# cuando tenga los demas parametros listos, agregar (, embedding_model = 'paraphrase-multilingual-mpnet-base-v2')
# topic_model = BERTopic(language="multilingual", umap_model=umap_model, hdbscan_model=hdbscan_model, calculate_probabilities=True, verbose=True, n_gram_range = (1, 3), nr_topics = 'auto', )
topic_model = BERTopic(language="multilingual", hdbscan_model=hdbscan_model, umap_model=umap_model, calculate_probabilities=True, verbose=True, n_gram_range = (1, 3) , embedding_model='all-mpnet-base-v2')
topics, probs = topic_model.fit_transform(docs)
freq = topic_model.get_topic_info(); freq.head(5)

Batches:   0%|          | 0/135 [00:00<?, ?it/s]

2022-02-14 12:08:11,369 - BERTopic - Transformed documents to Embeddings
2022-02-14 12:08:28,493 - BERTopic - Reduced dimensionality with UMAP
2022-02-14 12:08:30,145 - BERTopic - Clustered UMAP embeddings with HDBSCAN


,Topic,Count,Name
0,-1,1061,-1_que_no_la_me
1,0,356,0_cal_cal center_cal centre_centre
2,1,268,1_whatsapp_por whatsapp_por_que
3,2,221,2_call_call center_center_call center no
4,3,196,3_internet_el_problema_no


In [ ]:
topic_model.get_representative_docs(51)

['probelmas de señal , de internet , la app no funciona muy bien ,no entregue bien toda la información . lo informo en algún minuto ,pero no recuerdo bien .si es u llame no recuerdo.',
 'la señal se ha caído; si lo informé, dejé por la app hay opción de dejar mensajes. no respondieron nada.',
 'la app no me descarga, no me deja ver el consumo .no']

In [ ]:
topic_model.visualize_topics()

In [ ]:
df_large = df
df_large['topics'] = topics

In [ ]:
n = probs.shape[1]

col_names = []
for i in range(n):
  col_names += [f'Topic {i}']

In [ ]:
df_probs_large = pd.DataFrame(probs, columns = col_names)
df_large = df_large.merge(df_probs_large, left_index = True, right_index = True)

In [ ]:
df_large[col_names].aggregate({sum})

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,Topic 16,Topic 17,Topic 18,Topic 19,Topic 20,Topic 21,Topic 22,Topic 23,Topic 24,Topic 25,Topic 26,Topic 27,Topic 28,Topic 29,Topic 30,Topic 31,Topic 32,Topic 33,Topic 34,Topic 35,Topic 36,Topic 37,Topic 38,Topic 39,Topic 40,Topic 41,Topic 42,Topic 43,Topic 44,Topic 45,Topic 46,Topic 47,Topic 48,Topic 49,Topic 50,Topic 51,Topic 52,Topic 53,Topic 54,Topic 55,Topic 56,Topic 57,Topic 58,Topic 59,Topic 60,Topic 61,Topic 62,Topic 63,Topic 64,Topic 65,Topic 66,Topic 67
sum,220.778203,131.55034,116.818143,109.44987,124.060376,45.027268,56.431086,62.074276,44.528444,37.769061,43.491952,42.916484,43.340329,46.671455,38.824645,60.082581,38.933743,50.527392,44.266649,55.920601,53.954057,39.402667,43.613697,35.738632,48.225781,43.737786,41.059207,48.367262,42.778889,43.764563,41.014376,36.28971,38.767341,36.286401,40.628493,38.370814,37.25754,38.317702,41.08351,35.51608,34.384783,46.828853,42.073678,46.148356,45.584229,44.352955,38.307148,33.734764,36.536056,45.615021,34.184588,37.12032,37.667224,32.018493,44.725275,40.682387,41.597501,38.027694,45.949096,37.210335,38.905298,45.023351,41.036146,43.047409,33.397744,39.87338,40.160523,45.279611


In [ ]:
freq.head(n).transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,15,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,33,34,32,35,36,37,38,39,40,41,42,44,45,43,46,47,48,49,50,53,51,52,54,55,56,57,58,59,60,61,62,63,64,65,66,67
Topic,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,33,31,32,34,35,36,37,38,39,40,41,44,42,43,45,46,47,48,49,51,52,50,53,54,55,56,57,58,59,60,61,62,63,64,65,66
Count,1061,356,268,221,196,191,101,79,78,77,72,69,64,59,58,57,57,54,51,51,50,48,46,45,40,39,38,38,37,33,32,31,28,28,28,27,26,24,24,23,23,20,19,18,18,18,17,16,16,15,15,14,14,14,13,13,12,12,12,11,11,11,11,11,11,10,10,10
Name,-1_que_no_la_me,0_cal_cal center_cal centre_centre,1_whatsapp_por whatsapp_por_que,2_call_call center_center_call center no,3_internet_el_problema_no,4_teléfono_no_me_el teléfono,5_boleta_la boleta_me_boletas,6_plan_el plan_plan no_me,7_chip_el chip_el_que,8_llamadas_las llamadas_llamadas no_las llamad...,9_señal_la señal_señal no_la señal no,10_señal no_señal_la señal_no informe,11_pagina_paginas_la pagina_las paginas,12_internet_señal_la señal_el internet,13_señal_la señal_la_no,14_4g_3g_5g_el 4g,15_que_internet_es_se,16_red_la red_red no_no,17_103_al 103_al_me,18_103_al 103_internet_el,19_mil_plan_pesos_me,20_servicio_el servicio_cliente_al cliente,21_youtube_vídeos_no_vídeo,22_comunicar_comunicarme_comunicación_no,23_conectividad_la conectividad_de conectivida...,24_no lo_he informado_internet_informado,25_problema_señal_el problema_la señal,26_movistar_con movistar_movistar no_no,27_plan_teléfono_el plan_me,28_internet_el internet_internet no_no lo,29_entel_de entel_entel no_de,30_pago_pague_que_pagar,33_cobertura_cobertura no_cobertura no lo_la c...,31_redes sociales_sociales_redes_las redes soc...,32_sector_señal_donde_el sector,34_gigas_me_los gigas_que,35_el internet_internet no_el internet no_inte...,36_datos_los datos_datos móviles_móviles,37_compañia_cobertura_empresa_la,38_llamadas_la llamada_llamar_llamada,39_navegación_la navegación_navegar_navegación no,40_planes_dos planes_los planes_los,41_equipo_el equipo_el_equipo no,44_dieron_internet_me dieron_no me dieron,42_cobertura_no hay cobertura_hay cobertura_hay,43_casa_señal_la señal_mi casa,45_señal_la señal_hay_la,46_mil_boleta_pesos_mil pesos,47_cobertura_cobertura no la_cobertura no_la c...,48_facebook_no_el facebook_solución,49_yo_que_plan_me,51_app_la app_app no_la app no,52_movistar_internet_el internet_el,50_tema de_tema_el tema de_el tema,53_señal no la_no la_no la señal_señal no,54_problema_problema de_el problema_problema d...,55_de internet no_internet no_señal de interne...,56_plan_internet_el plan_un plan,57_lento_internet_lento internet_esta lento,58_internet_el internet_se_no lo,59_cobertura_cobertura no_la cobertura_la cobe...,60_21_pago_los 21_la fecha,61_me_plan_cobraron_me cobraron,62_numero_el numero_tenia_plan,63_cobertura_de la cobertura_la cobertura_el p...,64_canal_ningún canal_canales_no informo ningún,65_solucion me_me estaban cobrando_cobrando_es...,66_cobro_cobros_monto_del cobro no


In [ ]:
factor = .05
for t in col_names:
    df_large.loc[:, t] = df_large[t].apply(lambda x: 1 if x >= factor else 0)

In [ ]:
#sacamos el promedio general
avg = df_large[['REC_CIA']].apply({np.mean})
print(round(avg))

      REC_CIA
mean    -16.0


In [ ]:
# Load model
topic_model = BERTopic.load("my_model")

In [ ]:
with open('topics', 'rb') as f:
    topics = pickle.load(f)

with open('probs', 'rb') as f:
    probs = pickle.load(f)

In [ ]:
df_large = pd.read_excel('df_large.xlsx')
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,-1,1061,-1_que_no_la_me
1,0,356,0_cal_cal center_cal centre_centre
2,1,268,1_whatsapp_por whatsapp_por_que
3,2,221,2_call_call center_center_call center no
4,3,196,3_internet_el_problema_no


In [ ]:
def info_topics(topic):
  t = topic
  # Imprime las palabras clave del topico
  for i in range (5):
    print(topic_model.get_topic(t)[i][0], end = ', ')
  print('')

  # Imprime los comentarios mas representativos
  print('')
  nro_comm = len(topic_model.get_representative_docs(t))
  for n in range(nro_comm):
    print(topic_model.get_representative_docs(t)[n])

    # Imprime la cuenta
  print('')
  print('N: ' + str(freq['Count'][t+1]))

  # Imprime el promedio de rec_cia
  avg = df_large[['REC_CIA']].apply({np.mean}).values[0][0]
  avg_t = df_large[df_large[f'Topic {t}'] == 1][['REC_CIA']].apply({np.mean}).values[0][0]
  print('Delta: ' + str(round(avg - avg_t)))

  return

In [ ]:
for i in range(66):
  print("")
  print(f"TOPIC {i}")
  info_topics(i)


topic 0
cal, cal center, cal centre, centre, center, 

la navegación  y tratarme de contactar con el cal center, si; pero no por un tiempo si no por un par de días que todo volvió a estar igual
el tema del sin servicio; que hay días que no tengo servicio y es que vivo en el centro de santiago. si lo informé al cal centre pero no he tenido solución.
se cae mucho la señal. si, cal center, no

N: 356
Delta: 5

topic 1
whatsapp, por whatsapp, por, que, de, 

cobro excesivo, si varias veces, por whatsapp, supuesta mente me anularon los cargos extras que venian
con las cobranzas; que de repente no llega el mes y ya me están cobrando , quitando los gigas. si, por whatsapp. no.
señal; se pierde, no mandan la calidad. si lo informé a través de whatsapp que tienen ellos pero no hay respuesta.

N: 268
Delta: -5

topic 2
call, call center, center, call center no, center no, 

cuando no hay señal nadie le responde y no le responde, por el call center y me derivan a varias personas y no le solucion

In [ ]:
top_n_topics = probs.shape[1]
topic_model.visualize_hierarchy(top_n_topics=top_n_topics)

In [ ]:
info_topics(15)

que, internet, es, se, de, 

muy mala la señal de internet ya e llamado y dicen que lo solucionan y no solucionan nada llamo al servicio al cliente dijeron que iban a mejorar y no mejoro nada
eh mala señal de internet sin cobertura, que no tengo internet tengo que mandar mensaje hacer llamada y todo, llame al cal centre del servicio al cliente pero ahora te atiende una maquina no puedo hablar con nadie es una operadora virtual una ejecutiva, dice que secretea la señal y corta  ok respetamos la señal y corta no hay ningún feedback
se cae la señal bastante en internet y a veces en llamadas. sí, informé problemas a través de servicio al cliente. dieron solución, pero, se tardaron bastante.

N: 57
Delta: 15


In [ ]:
df_large['total'] = 0

for c in col_names:
  df_large['total'] += df_large[t]

len(df_large[df_large['total'] > 0])

39

In [ ]:
df_aux = pd.DataFrame()

n_topics = probs.shape[1]

comm = []
for c in range(3):
  df_aux[f'Comm_{c}'] = ''
  comm += [f'Comm_{c}']

for n in range(n_topics):
  df_aux.loc[n, 'Topic'] = f'Topic {n}'

  for c in range(3):
      df_aux.loc[n, comm[c]] = topic_model.get_representative_docs(n)[c]

In [ ]:
for t in range(n_topics):
  df_aux.loc[t, 'Sat'] = int(round(df_large[df_large[f'Topic {t}'] == 1][['REC_CIA']].apply({np.mean}).values[0][0]))

In [ ]:
df_aux.to_excel('df_aux.xlsx')

In [ ]:
fig = topic_model.visualize_topics()
fig.write_html("topics_model.html")

fig = topic_model.visualize_hierarchy(top_n_topics=top_n_topics)
fig.write_html("Clusters_Hierarchy.html")


In [ ]:
macro_topics = dict()

# internet, cobertura
internet = [0,2,3,4,9,10,11,12,13,15,20,22,23,24,25,28,28,32,33,35,36,37,39,41,42,43,44,45,47,50,53,54,55,56,57,58,59,62,63,67,14,21,48,51,31,1]

# cobros
cobros = [66,30,60,65,5,46,19,40,61,6,49]

# LLamadas
llamadas = [8,38,16]

# Plan
plan = [34,27]

#Equipo
equipo = [7]

#Compania
compania = [26,52,29]

#Canal
canal = [64,17,18]


macro_topics_keys = ['internet', 'cobros', 'llamadas', 'plan', 'equipo', 'compania', 'canal']
macro_topics_values = [internet, cobros, llamadas, plan, equipo, compania, canal]
l = len(macro_topics_values)


for i in range(l):
    macro_topics.update(dict.fromkeys(macro_topics_values[i], macro_topics_keys[i]))

In [ ]:
df_large[macro_topics_keys] = 0 

In [ ]:
for i in range(len(df_large)):
  for c in col_names:
    ti = len(c) - 6
    t = int(c[-ti:])
    col = macro_topics[t]

    df_large.loc[i, col] += df_large.loc[i, c]

In [ ]:
df_large.to_excel('df_large.xlsx')

In [ ]:
with open('probs', 'wb') as f:
    pickle.dump(probs, f)
    
with open('topics', 'wb') as f:
    pickle.dump(topics, f)

In [ ]:
# Save model
topic_model.save("my_model")	

In [ ]:
with open('topics', 'rb') as f:
    topics = pickle.load(f)

with open('probs', 'rb') as f:
    probs = pickle.load(f)

In [ ]:
topic_model = BERTopic.load('my_model')

In [ ]:
topics_mini, probs_mini = topic_model.reduce_topics(docs, topics, probs, nr_topics=10)

2022-02-11 16:33:27,651 - BERTopic - Reduced number of topics from 69 to 11


In [ ]:
freq_mini = topic_model.get_topic_info(); freq_mini.head(15)

,Topic,Count,Name
0,-1,2495,-1_no_que_la_me
1,0,356,0_cal_cal center_no_cal centre
2,1,298,1_internet_no_el_el internet
3,2,268,2_whatsapp_que_no_por
4,3,221,3_call_call center_center_de
5,4,191,4_teléfono_no_me_que
6,5,138,5_señal_no_la_cobertura
7,6,101,6_boleta_me_la boleta_que
8,7,85,7_señal_la señal_señal no_la
9,8,79,8_plan_el plan_me_el


In [ ]:
topic_model.get_representative_docs(9)

['de señal móvil el internet del teléfono que se cae a cada rato tiene que estar constantemente reiniciando y se cae el internet y tengo que reiniciar si lo informe al cal centre en 3 oportunidades que reinicie el equipo cambiamos el chip pero mas es el problema de la señal',
 'pedi que me cambiaran el chip para ver si era el chip por la cobertura y quedaron de ir a dejármelo y nunca fueron no no lo informe para que perder el tiempo',
 'no tenia el chip no funcionaba no tenia señal y cada vez que me realizaban llamadas me salia que el teléfono estaba inactivo en la oficina si me cambiaron el chip']

In [ ]:
topic_model.visualize_topics()

In [ ]:
fig = topic_model.visualize_topics()
fig.write_html("topics_model_mini.html")

In [ ]:
with open('probs_mini', 'wb') as f:
    pickle.dump(probs_mini, f)
    
with open('topics_mini', 'wb') as f:
    pickle.dump(topics_mini, f)

In [ ]:
# Save model
topic_model.save("my_model_mini")	

In [ ]:
df_mini = df
df_mini['topics'] = topics_mini

In [ ]:
n = probs_mini.shape[1]

col_names = []
for i in range(n):
  col_names += [f'Topic {i}']

In [ ]:
df_probs_mini = pd.DataFrame(probs_mini, columns = col_names)
df_probs_mini.head(5)

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9
0,0.006008,6.280514e-02,9.124547e-03,1.072179e-02,2.145339e-02,2.858232e-02,1.008048e-02,9.887919e-03,1.645131e-02,1.120414e-02
1,1.000000,2.195220e-307,4.340526e-308,4.834678e-308,5.497992e-308,1.802187e-307,7.976642e-308,6.595847e-308,7.204085e-308,6.312580e-308
2,0.005439,1.884227e-02,3.452881e-03,5.373469e-03,7.022084e-03,1.429085e-02,1.126695e-02,4.997240e-03,2.720397e-02,4.782067e-03
3,0.002606,1.082564e-01,4.679149e-03,3.816891e-03,7.589098e-03,2.098909e-02,3.324620e-03,7.518380e-03,3.613934e-03,4.025061e-03
4,0.004607,1.499209e-02,2.593030e-03,3.884507e-03,5.257481e-03,1.461918e-02,6.053572e-02,4.971161e-03,6.952332e-03,3.960538e-03


In [ ]:
df_mini = df_mini.merge(df_probs_mini, left_index = True, right_index = True)

In [ ]:
factor = .05
for t in col_names:
    df_mini.loc[:, t] = df_mini[t].apply(lambda x: 1 if x >= factor else 0)

In [ ]:
df_mini.to_excel('df_mini.xlsx')